# Descriptors

Classes that implement the descriptor protocol and allow specialised behaviour when accessed.

The descriptor interface includes the following methods:
* `__get__()`
* `__set__()`
* `__delete__()`
* `__set_name__()`

If class only implements `__get__` then it is a non-data descriptor, else it is called a data-descriptor.

Using the non-data descriptor...

In [1]:
import types
import inspect

class MyNonDataDescriptor:
    def __init__(self):
        print("MyNonDataDescriptor initialisation")

    def __get__(self, obj, type=None):                          # self is MyNonDataDescriptor instance
        print("__get__ called in MyNonDataDescriptor")          # and obj is the object we're attached to
        print("__get__ obj=", obj, "type=", type)               # ...you'll get a MyClass obj in this case
        return 19

class MyDataDescriptor:
    def __init__(self):
        print("MyDataDescriptor initialisation")
        self.some_value = 'some_value'

    def __get__(self, obj, type=None):
        """implements the __get__ method of the descriptor protocol"""
        print("__get__ called in MyDataDescriptor")
        return self.some_value

    def __set__(self, obj, value):
        """implements the __set__ method of the descriptor protocol"""
        print("__set__ called in MyDataDescriptor")
        self.some_value = value

class MyReadOnlyDescriptor:
    def __init__(self):
        print("MyReadOnlyDescriptor initialisation")

    def __get__(self, obj, type=None):
        print("__get__ called in MyReadOnlyDescriptor")
        return 19

    def __set__(self, obj, value):
        print("__set__ called in MyReadOnlyDescriptor")
        raise AttributeError("this value is read only!")

class MyClass:
    # once instantiated as part of MyClass it can considered a descriptor
    # as can be accessed using the dot (.) operator which these descriptors override and have precedence
    print("descriptor objects live in the class and not the instance")
    my_descriptor = MyNonDataDescriptor()
    my_data_descriptor = MyDataDescriptor()
    my_ro_descriptor = MyReadOnlyDescriptor()

    def __init__(self):
        pass

my_instance = MyClass()

print("type(my_instance.my_descriptor) =", type(my_instance.my_descriptor))
print("my_instance.my_descriptor at", id(my_instance.my_descriptor))
print("my_instance.my_descriptor =", my_instance.my_descriptor)
my_instance.my_descriptor = 'my new value'                                      # no __set__ method so...
print("type(my_instance.my_descriptor) =", type(my_instance.my_descriptor))     # no longer a descriptor, just a string
print("my_instance.my_descriptor at", id(my_instance.my_descriptor))            # __get__ no longer called
print("my_instance.my_descriptor =", my_instance.my_descriptor)

descriptor objects live in the class and not the instance
MyNonDataDescriptor initialisation
MyDataDescriptor initialisation
MyReadOnlyDescriptor initialisation
__get__ called in MyNonDataDescriptor
__get__ obj= <__main__.MyClass object at 0x000001401D6D3BE0> type= <class '__main__.MyClass'>
type(my_instance.my_descriptor) = <class 'int'>
__get__ called in MyNonDataDescriptor
__get__ obj= <__main__.MyClass object at 0x000001401D6D3BE0> type= <class '__main__.MyClass'>
my_instance.my_descriptor at 1374798807856
__get__ called in MyNonDataDescriptor
__get__ obj= <__main__.MyClass object at 0x000001401D6D3BE0> type= <class '__main__.MyClass'>
my_instance.my_descriptor = 19
type(my_instance.my_descriptor) = <class 'str'>
my_instance.my_descriptor at 1374882247344
my_instance.my_descriptor = my new value


Using the data descriptor.

In [2]:
print("type(my_instance.my_data_descriptor) =", type(my_instance.my_data_descriptor))
print("my_instance.my_data_descriptor at", id(my_instance.my_data_descriptor))
print("my_instance.my_data_descriptor =", my_instance.my_data_descriptor)
my_instance.my_data_descriptor = 'ive changed this value'
print("type(my_instance.my_data_descriptor) =", type(my_instance.my_data_descriptor))   # remains MyDataDescriptor
print("my_instance.my_data_descriptor at", id(my_instance.my_data_descriptor))          # __get__ still called
print("my_instance.my_data_descriptor =", my_instance.my_data_descriptor)

__get__ called in MyDataDescriptor
type(my_instance.my_data_descriptor) = <class 'str'>
__get__ called in MyDataDescriptor
my_instance.my_data_descriptor at 1374882288176
__get__ called in MyDataDescriptor
my_instance.my_data_descriptor = some_value
__set__ called in MyDataDescriptor
__get__ called in MyDataDescriptor
type(my_instance.my_data_descriptor) = <class 'str'>
__get__ called in MyDataDescriptor
my_instance.my_data_descriptor at 1374894390880
__get__ called in MyDataDescriptor
my_instance.my_data_descriptor = ive changed this value


Using the read-only descriptor...

In [3]:
print("type(my_instance.my_ro_descriptor) =", type(my_instance.my_ro_descriptor))
print("my_instance.my_ro_descriptor at", id(my_instance.my_ro_descriptor))
print("my_instance.my_ro_descriptor =", my_instance.my_ro_descriptor)
try:
    my_instance.my_ro_descriptor = 'ive changed this value'                     # recommended way to implement
except AttributeError as e:                                                     # read-only attribute
    print("exception handler:", e)
print("type(my_instance.my_ro_descriptor) =", type(my_instance.my_ro_descriptor))
print("my_instance.my_ro_descriptor at", id(my_instance.my_ro_descriptor))
print("my_instance.my_ro_descriptor =", my_instance.my_ro_descriptor)


__get__ called in MyReadOnlyDescriptor
type(my_instance.my_ro_descriptor) = <class 'int'>
__get__ called in MyReadOnlyDescriptor
my_instance.my_ro_descriptor at 1374798807856
__get__ called in MyReadOnlyDescriptor
my_instance.my_ro_descriptor = 19
__set__ called in MyReadOnlyDescriptor
exception handler: this value is read only!
__get__ called in MyReadOnlyDescriptor
type(my_instance.my_ro_descriptor) = <class 'int'>
__get__ called in MyReadOnlyDescriptor
my_instance.my_ro_descriptor at 1374798807856
__get__ called in MyReadOnlyDescriptor
my_instance.my_ro_descriptor = 19


What is self?

In [4]:
class MySelf:
    def __init__(self):
        print("MySelf initialisation")
        print("type(self) =", type(self))
        print("id(self) =", id(self))
        print(self)                                     # what is self?, self is a MySelf object
        self.some_attribute = 'some instance attribute'

    def do_something(funky_chicken, some_parameter):    # you don't need to call it 'self' - it's just the
        print("in do_something function")               # ...accepted name to use for first parameter in bound method
        print("type(funky_chicken) =", type(funky_chicken))
        print("id(funky_chicken) =", id(funky_chicken))
        print("funky_chicken.some_attribute =", funky_chicken.some_attribute)
        print("type(some_parameter) =", type(some_parameter))
        print("id(some_parameter) =", id(some_parameter))
        if some_parameter is funky_chicken:
            print("hey we've passed self twice!")
        else:
            print(some_parameter)

my_instance = MySelf()
my_instance.do_something('hello')
my_instance.do_something(my_instance)

class MyClass:
    my_class_variable = 1                   # not a descriptor, will appear in MyClass.__dict__

    def __init__(self):
        self.my_instance_variable = 2       # will appear in my_instance.__dict__

MySelf initialisation
type(self) = <class '__main__.MySelf'>
id(self) = 1374883222080
in do_something function
type(funky_chicken) = <class '__main__.MySelf'>
id(funky_chicken) = 1374883222080
funky_chicken.some_attribute = some instance attribute
type(some_parameter) = <class 'str'>
id(some_parameter) = 1374874988400
hello
in do_something function
type(funky_chicken) = <class '__main__.MySelf'>
id(funky_chicken) = 1374883222080
funky_chicken.some_attribute = some instance attribute
type(some_parameter) = <class '__main__.MySelf'>
id(some_parameter) = 1374883222080
hey we've passed self twice!


What is `__dict__` attribute and the look up chain?

dict's are used if `__get__` method fails on any matching named data descriptors in class e.g. if I call print(`my_instance.my_class_variable`) interpreter will see if my_class_variable is a data descriptor that exposes the `__get__` interface; if that fails then it will try `my_instance.__dict__['my_class_variable']` and after that the lookup chain will look for a non-data descriptors with a `__get__` method, followed by type `(my_instance).__dict__['my_class_variable']` (or `MyClass.__dict__['my_class_variable']`) and failing all that will start progressing up parent classes

In [5]:
my_instance = MyClass()
print(my_instance.__dict__)
print(MyClass.__dict__)
print(type(my_instance).__dict__)

{'my_instance_variable': 2}
{'__module__': '__main__', 'my_class_variable': 1, '__init__': <function MyClass.__init__ at 0x000001401D5EBF40>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'__module__': '__main__', 'my_class_variable': 1, '__init__': <function MyClass.__init__ at 0x000001401D5EBF40>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


Dynamically adding a method to an object instance...

In [6]:
def my_dynamic_method(self, x):
    print("self.my_instance_variable + x =", self.my_instance_variable + x)

# monkey patch = modification/addition of attributes at run-time
my_instance.my_method = types.MethodType(my_dynamic_method, my_instance)
my_instance.my_method(5)            # method only available to this instance
del my_instance.my_method

self.my_instance_variable + x = 7


Use descriptor object to call instance method...

In [7]:
class MyFunctionDescriptor:
    def __init__(self):
        pass

    def __get__(self, obj, type=None):
        return obj.do_something

    def __set__(self, obj, value):
        print("__set__ called in MyReadOnlyDescriptor")
        raise AttributeError("this value is read only!")


class MyDynamicFunctionDescriptor:
    def __init__(self):
        pass

    def __get__(self, obj, type=None):
        return types.MethodType(my_dynamic_method, obj)     # similar to how instance methods are called

    def __set__(self, obj, value):
        print("__set__ called in MyReadOnlyDescriptor")
        raise AttributeError("this value is read only!")


class MySelf:
    def __init__(self):
        print("MySelf initialisation")
        print("type(self) =", type(self))
        print("id(self) =", id(self))
        print(self)                                     # what is self?, self is a MySelf object
        self.some_attribute = 'some instance attribute'
        self.my_instance_variable = 2

    def do_something(funky_chicken, some_parameter):    # you don't need to call it 'self' - it's just the
        print("in do_something function")               # ...accepted name to use for first parameter in bound method
        print("type(funky_chicken) =", type(funky_chicken))
        print("id(funky_chicken) =", id(funky_chicken))
        print("funky_chicken.some_attribute =", funky_chicken.some_attribute)
        print("type(some_parameter) =", type(some_parameter))
        print("id(some_parameter) =", id(some_parameter))
        if some_parameter is funky_chicken:
            print("hey we've passed self twice!")
        else:
            print(some_parameter)

    my_do_something = MyFunctionDescriptor()
    my_dynamic_something = MyDynamicFunctionDescriptor()

my_instance = MySelf()
my_instance.my_do_something('how odd')
my_instance.my_dynamic_something(19)

MySelf initialisation
type(self) = <class '__main__.MySelf'>
id(self) = 1374882603760
in do_something function
type(funky_chicken) = <class '__main__.MySelf'>
id(funky_chicken) = 1374882603760
funky_chicken.some_attribute = some instance attribute
type(some_parameter) = <class 'str'>
id(some_parameter) = 1374883166576
how odd
self.my_instance_variable + x = 21


To get descriptors to work with instances, not just classes...

In [8]:
class MyInstanceDescriptor:
    def __set_name__(self, owner, name):                # available since Python 3.6
        print("MyInstanceDescriptor __set_name__")
        print("owner =", owner)
        print("name =", name)
        self.name = name

    def __get__(self, obj, type=None):
        print("MyInstanceDescriptor __get__")
        return obj.__dict__.get(self.name) or 0

    def __set__(self, obj, value):
        print("MyInstanceDescriptor __set__")
        obj.__dict__[self.name] = value                 # store value in object instance dictionary

class MyClass:
    print("create instance descriptor in MyClass")
    my_instance_descriptor = MyInstanceDescriptor()

    def __init__(self):
        pass

my_instance1 = MyClass()
my_instance2 = MyClass()

print("MyClass.__dict__ =", MyClass.__dict__)
print("my_instance1.__dict__ =", my_instance1.__dict__)
print("my_instance2.__dict__ =", my_instance2.__dict__)
# MyClass.my_instance_descriptor = 3                        # this overwrites the descriptor with the 3 object..
my_instance1.my_instance_descriptor = 9                     # ...rather than call __set__ with obj = class object
my_instance2.my_instance_descriptor = 14
print("MyClass.__dict__ =", MyClass.__dict__)
print("my_instance1.__dict__ =", my_instance1.__dict__)
print("my_instance2.__dict__ =", my_instance2.__dict__)

create instance descriptor in MyClass
MyInstanceDescriptor __set_name__
owner = <class '__main__.MyClass'>
name = my_instance_descriptor
MyClass.__dict__ = {'__module__': '__main__', 'my_instance_descriptor': <__main__.MyInstanceDescriptor object at 0x000001401D6D0940>, '__init__': <function MyClass.__init__ at 0x000001401D72C5E0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
my_instance1.__dict__ = {}
my_instance2.__dict__ = {}
MyInstanceDescriptor __set__
MyInstanceDescriptor __set__
MyClass.__dict__ = {'__module__': '__main__', 'my_instance_descriptor': <__main__.MyInstanceDescriptor object at 0x000001401D6D0940>, '__init__': <function MyClass.__init__ at 0x000001401D72C5E0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
my_instance1.__dict__ = {'my_instance_descriptor': 9}
my_instance2.__dict__ = {'my_